# Skripsi v1

## Integrasi SMOTE dan Neural Network

In [1]:
def repro():
  seed_value= 42

  import os
  os.environ['PYTHONHASHSEED']=str(seed_value)
  import random
  random.seed(seed_value)

  import numpy as np
  np.random.seed(seed_value)

  import tensorflow as tf
  tf.random.set_seed(seed_value)

  from tensorflow.keras import backend as K
  
  session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
  sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
  tf.compat.v1.keras.backend.set_session(sess)

In [2]:
repro()

In [3]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

# Import 

In [4]:
import model
import metric
import pandas as pd

# import 

from sklearn.preprocessing import MinMaxScaler
from bayes import BayesianOptimization as bayes
import tool

# Dataset

In [5]:
dataset_kc1 = pd.read_csv("../Dataset/clean/KC1.csv")
dataset_kc2 = pd.read_csv("../Dataset/clean/KC2.csv")
dataset_pc1 = pd.read_csv("../Dataset/clean/PC1.csv")
dataset_pc3 = pd.read_csv("../Dataset/clean/PC3.csv")
dataset_pc4 = pd.read_csv("../Dataset/clean/PC4.csv")
dataset_cm1 = pd.read_csv("../Dataset/clean/CM1.csv")
dataset_jm1 = pd.read_csv("../Dataset/clean/JM1.csv")

In [6]:
dataset_jm1.loc[dataset_jm1['uniq_Op'] == '?']

,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defects
143,49.0,4.0,3.0,3.0,185.0,1097.19,0.05,18.84,58.25,20667.93,...,38,0,9,0,?,?,?,?,?,True
358,27.0,5.0,3.0,3.0,79.0,417.55,0.07,13.52,30.88,5646.37,...,19,0,6,0,?,?,?,?,?,True
1598,76.0,18.0,11.0,11.0,275.0,1650.00,0.03,34.50,47.83,56925.00,...,62,0,12,0,?,?,?,?,?,True
4214,6.0,1.0,1.0,1.0,15.0,51.89,0.34,2.92,17.79,151.35,...,3,0,1,0,?,?,?,?,?,False
8279,9.0,13.0,10.0,4.0,0.0,0.00,0.00,0.00,0.00,0.00,...,0,0,0,0,?,?,?,?,?,False


In [7]:
idx_missing_JM1 = dataset_jm1.loc[dataset_jm1['uniq_Op'] == '?'].index
dataset_jm1 = dataset_jm1.drop(idx_missing_JM1)

In [8]:
dataset = {
    'KC1' : dataset_kc1,
    'KC2' : dataset_kc2,
    'PC1' : dataset_pc1,
    'PC3' : dataset_pc3,
    'PC4' : dataset_pc4,
    'CM1' : dataset_cm1,
    'JM1' : dataset_jm1,
}

Setiap dataset memiliki column class berbeda-beda. dibawah ini proses mengubah kolom kelas menjadi seragam ("Defect")

In [9]:
for i in dataset:
    print(i)
    dataset[i] = dataset[i].rename(columns = {dataset[i].columns[-1]:'defect'})
    display(dataset[i])

KC1


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defect
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2,2,2,2,1.2,1.2,1.2,1.2,1.4,False
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,True
2,83.0,11.0,1.0,11.0,171.0,927.89,0.04,23.04,40.27,21378.61,...,65,10,6,0,18.0,25.0,107.0,64.0,21.0,True
3,46.0,8.0,6.0,8.0,141.0,769.78,0.07,14.86,51.81,11436.73,...,37,2,5,0,16.0,28.0,89.0,52.0,15.0,True
4,25.0,3.0,1.0,3.0,58.0,254.75,0.11,9.35,27.25,2381.95,...,21,0,2,0,11.0,10.0,41.0,17.0,5.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2104,19.0,2.0,1.0,2.0,40.0,175.69,0.15,6.82,25.77,1197.90,...,12,1,2,0,10.0,11.0,25.0,15.0,3.0,False
2105,23.0,3.0,3.0,3.0,60.0,278.63,0.10,9.69,28.75,2700.58,...,18,1,2,0,12.0,13.0,39.0,21.0,5.0,False
2106,2.0,1.0,1.0,1.0,4.0,8.00,0.67,1.50,5.33,12.00,...,0,0,0,0,3.0,1.0,3.0,1.0,1.0,False
2107,13.0,1.0,1.0,1.0,17.0,60.94,0.25,4.00,15.24,243.78,...,6,0,5,0,6.0,6.0,9.0,8.0,1.0,False


KC2


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defect
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2,2,2,2,1.2,1.2,1.2,1.2,1.4,False
1,3.0,1.0,1.0,1.0,1.0,0.00,0.00,0.00,0.00,0.00,...,1,0,0,0,1.0,0.0,1.0,0.0,1.0,False
2,14.0,2.0,1.0,2.0,22.0,88.00,0.17,5.79,15.21,509.14,...,8,0,1,0,9.0,7.0,13.0,9.0,3.0,False
3,10.0,2.0,1.0,2.0,18.0,64.53,0.14,7.00,9.22,451.71,...,8,0,0,0,8.0,4.0,11.0,7.0,3.0,False
4,8.0,1.0,1.0,1.0,10.0,31.70,0.50,2.00,15.85,63.40,...,3,0,0,1,4.0,5.0,5.0,5.0,1.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,4.0,1.0,1.0,1.0,5.0,11.61,0.50,2.00,5.80,23.22,...,2,0,0,0,4.0,1.0,4.0,1.0,1.0,True
518,4.0,1.0,1.0,1.0,4.0,8.00,0.67,1.50,5.33,12.00,...,2,0,0,0,3.0,1.0,3.0,1.0,1.0,True
519,4.0,1.0,1.0,1.0,4.0,8.00,0.67,1.50,5.33,12.00,...,2,0,0,0,3.0,1.0,3.0,1.0,1.0,True
520,4.0,1.0,1.0,1.0,5.0,11.61,0.67,1.50,7.74,17.41,...,2,0,0,0,3.0,2.0,3.0,2.0,1.0,True


PC1


...

,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,...,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,defect
0,17,11,5,2,8,20,6,0.25,10,2,...,25,0.11,53,49,23,12,57,31.25,24,False
1,2,9,3,0,1,16,5,0.56,6,2,...,14,0.36,13,24,7,14,14,10.00,9,False
2,2,5,1,1,1,6,3,0.17,2,3,...,7,0.13,16,28,9,14,23,10.53,18,False
3,4,5,1,0,0,8,3,0.30,4,2,...,10,0.19,13,16,10,9,16,0.00,10,False
4,7,5,1,3,0,0,3,0.15,0,0,...,10,0.11,26,46,7,7,28,15.00,20,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,1,7,2,0,0,0,4,0.40,0,0,...,14,0.31,13,35,6,10,13,0.00,10,False
1454,12,3,8,0,6,4,2,0.11,2,2,...,14,0.05,28,40,19,13,39,24.00,19,False
1455,0,1,0,0,0,0,1,1.00,0,0,...,2,1.00,0,2,0,2,1,0.00,0,False
1456,8,9,0,0,10,0,5,0.42,0,0,...,10,0.16,20,38,9,20,32,45.45,12,False


CM1


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defect
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2,2,2,2,1.2,1.2,1.2,1.2,1.4,False
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1.0,1.0,1.0,1.0,1.0,True
2,24.0,5.0,1.0,3.0,63.0,309.13,0.11,9.50,32.54,2936.77,...,1,0,6,0,15.0,15.0,44.0,19.0,9.0,False
3,20.0,4.0,4.0,2.0,47.0,215.49,0.06,16.00,13.47,3447.89,...,0,0,3,0,16.0,8.0,31.0,16.0,7.0,False
4,24.0,6.0,6.0,2.0,72.0,346.13,0.06,17.33,19.97,5999.58,...,0,0,3,0,16.0,12.0,46.0,26.0,11.0,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,47.0,3.0,1.0,3.0,256.0,1563.78,0.04,28.00,55.85,43785.90,...,2,13,2,0,23.0,46.0,144.0,112.0,5.0,True
494,24.0,4.0,3.0,3.0,107.0,587.63,0.05,19.13,30.72,11241.58,...,1,7,4,0,22.0,23.0,67.0,40.0,7.0,True
495,82.0,11.0,3.0,10.0,475.0,3155.83,0.02,44.71,70.59,141084.24,...,9,59,35,0,32.0,68.0,285.0,190.0,21.0,True
496,10.0,2.0,1.0,1.0,32.0,150.41,0.15,6.50,23.14,977.69,...,1,12,4,0,13.0,13.0,19.0,13.0,3.0,True


JM1


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defect
0,1.1,1.4,1.4,1.4,1.3,1.30,1.30,1.30,1.30,1.30,...,2,2,2,2,1.2,1.2,1.2,1.2,1.4,False
1,1.0,1.0,1.0,1.0,1.0,1.00,1.00,1.00,1.00,1.00,...,1,1,1,1,1,1,1,1,1,True
2,72.0,7.0,1.0,6.0,198.0,1134.13,0.05,20.31,55.85,23029.10,...,51,10,8,1,17,36,112,86,13,True
3,190.0,3.0,1.0,3.0,600.0,4348.76,0.06,17.06,254.87,74202.67,...,129,29,28,2,17,135,329,271,5,True
4,37.0,4.0,1.0,4.0,126.0,599.12,0.06,17.19,34.86,10297.30,...,28,1,6,0,11,16,76,50,7,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10880,18.0,4.0,1.0,4.0,52.0,241.48,0.14,7.33,32.93,1770.86,...,13,0,2,0,10,15,30,22,7,False
10881,9.0,2.0,1.0,2.0,30.0,129.66,0.12,8.25,15.72,1069.68,...,5,0,2,0,12,8,19,11,3,False
10882,42.0,4.0,1.0,2.0,103.0,519.57,0.04,26.40,19.68,13716.72,...,29,1,10,0,18,15,59,44,7,False
10883,10.0,1.0,1.0,1.0,36.0,147.15,0.12,8.44,17.44,1241.57,...,6,0,2,0,9,8,21,15,1,False


## Melihat Distribution

Melihat distribusi Dataset

In [10]:
# from sklearn.decomposition import PCA
import numpy as np
# from sklearn.preprocessing import StandardScaler
# import matplotlib.pyplot as plt
# %matplotlib inline
scaller = MinMaxScaler()


# pca = PCA(n_components=3)
# for i in dataset:
#     data = dataset[i]
#     x = data.loc[:, data.columns != 'defect'].values 
#     x = scaller.fit_transform(x)

    
#     principalComponents = pca.fit_transform(x)
#     principalDf = pd.DataFrame(data = principalComponents, columns = ['V1', 'V2','V3'])
#     finalDf = pd.concat([principalDf, data[['defect']]], axis = 1)
    
#     fig = plt.figure(figsize = (8,8))
#     ax = plt.axes(projection ='3d') 
#     ax.set_xlabel('V1', fontsize = 15)
#     ax.set_ylabel('V2', fontsize = 15)
#     ax.set_zlabel('V3', fontsize = 15) 
#     ax.set_title(i, fontsize = 20)
#     targets = [False, True]
#     colors = ['g', 'r','b']
#     for target, color in zip(targets,colors):
#         indicesToKeep = finalDf['defect'] == targets
#         ax.scatter(finalDf.loc[indicesToKeep, 'V1'], finalDf.loc[indicesToKeep, 'V2'],finalDf.loc[indicesToKeep, 'V3'], c = color, s = 50)
#         ax.legend(targets)
#         ax.grid()

# Check DMR

In [11]:
dmr_data = {}
for i in dataset:
    unique, counts = np.unique(dataset[i]['defect'], return_counts=True)
    dmr = (round((counts[1]/(counts[0]+counts[1])*100),2))
    print(dmr, unique, counts)
    dmr_data[i] = dmr

15.46 [False  True] [1783  326]
20.5 [False  True] [415 107]
6.94 [False  True] [1032   77]
12.44 [False  True] [943 134]
12.21 [False  True] [1280  178]
9.84 [False  True] [449  49]
19.33 [False  True] [8777 2103]


# Penerapan Min-Max Scaller untuk mengubah variant distribusi
$$ x_{scaled} = \frac{x - x_{min}}{x_{max} - x_{min}} $$

In [12]:

for i in dataset:
    dataset[i].loc[:, dataset[i].columns != dataset[i].columns[-1]] = scaller.fit_transform(dataset[i].loc[:, dataset[i].columns != dataset[i].columns[-1]])
#     display(dataset[i].loc[:, dataset[i].columns[0]:dataset[i].columns[4]].head(3))
    print(i)
    display(dataset[i])

KC1


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defect
0,0.000348,0.009091,0.016,0.009091,0.001175,0.000164,0.650,0.024186,0.006734,0.000004,...,0.007634,0.045455,0.034483,0.166667,0.032432,0.010000,0.001770,0.002804,0.004545,False
1,0.000000,0.000000,0.000,0.000000,0.000904,0.000126,0.500,0.018605,0.005180,0.000003,...,0.003817,0.022727,0.017241,0.083333,0.027027,0.008333,0.001475,0.002336,0.000000,True
2,0.285714,0.227273,0.000,0.227273,0.154611,0.117175,0.020,0.428651,0.208588,0.065820,...,0.248092,0.227273,0.103448,0.000000,0.486486,0.208333,0.157817,0.149533,0.227273,True
3,0.156794,0.159091,0.200,0.159091,0.127486,0.097209,0.035,0.276465,0.268362,0.035211,...,0.141221,0.045455,0.086207,0.000000,0.432432,0.233333,0.131268,0.121495,0.159091,True
4,0.083624,0.045455,0.000,0.045455,0.052441,0.032170,0.055,0.173953,0.141148,0.007334,...,0.080153,0.000000,0.034483,0.000000,0.297297,0.083333,0.060472,0.039720,0.045455,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2104,0.062718,0.022727,0.000,0.022727,0.036166,0.022186,0.075,0.126884,0.133482,0.003688,...,0.045802,0.022727,0.034483,0.000000,0.270270,0.091667,0.036873,0.035047,0.022727,False
2105,0.076655,0.045455,0.080,0.045455,0.054250,0.035186,0.050,0.180279,0.148917,0.008315,...,0.068702,0.022727,0.034483,0.000000,0.324324,0.108333,0.057522,0.049065,0.045455,False
2106,0.003484,0.000000,0.000,0.000000,0.003617,0.001010,0.335,0.027907,0.027608,0.000037,...,0.000000,0.000000,0.000000,0.000000,0.081081,0.008333,0.004425,0.002336,0.000000,False
2107,0.041812,0.000000,0.000,0.000000,0.015371,0.007696,0.125,0.074419,0.078939,0.000751,...,0.022901,0.000000,0.086207,0.000000,0.162162,0.050000,0.013274,0.018692,0.000000,False


KC2


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,lOCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defect
0,0.000078,0.002235,0.003226,0.002817,0.000075,0.000038,0.650,0.012557,0.003132,6.053597e-07,...,0.001807,0.045455,0.016529,0.181818,0.004348,0.003692,0.000081,0.000793,0.001111,False
1,0.001570,0.000000,0.000000,0.000000,0.000000,0.000000,0.000,0.000000,0.000000,0.000000e+00,...,0.000903,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,False
2,0.010204,0.005587,0.000000,0.007042,0.005275,0.002602,0.085,0.055926,0.036645,2.370868e-04,...,0.007227,0.000000,0.008264,0.000000,0.173913,0.021538,0.004862,0.005948,0.005556,False
3,0.007064,0.005587,0.000000,0.007042,0.004270,0.001908,0.070,0.067613,0.022214,2.103439e-04,...,0.007227,0.000000,0.000000,0.000000,0.152174,0.012308,0.004052,0.004627,0.005556,False
4,0.005495,0.000000,0.000000,0.000000,0.002261,0.000937,0.250,0.019318,0.038187,2.952293e-05,...,0.002710,0.000000,0.000000,0.090909,0.065217,0.015385,0.001621,0.003305,0.000000,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,0.002355,0.000000,0.000000,0.000000,0.001005,0.000343,0.250,0.019318,0.013974,1.081266e-05,...,0.001807,0.000000,0.000000,0.000000,0.065217,0.003077,0.001216,0.000661,0.000000,True
518,0.002355,0.000000,0.000000,0.000000,0.000754,0.000237,0.335,0.014489,0.012842,5.587935e-06,...,0.001807,0.000000,0.000000,0.000000,0.043478,0.003077,0.000810,0.000661,0.000000,True
519,0.002355,0.000000,0.000000,0.000000,0.000754,0.000237,0.335,0.014489,0.012842,5.587935e-06,...,0.001807,0.000000,0.000000,0.000000,0.043478,0.003077,0.000810,0.000661,0.000000,True
520,0.002355,0.000000,0.000000,0.000000,0.001005,0.000343,0.335,0.014489,0.018648,8.107163e-06,...,0.001807,0.000000,0.000000,0.000000,0.043478,0.006154,0.000810,0.001322,0.000000,True


PC1


...

,LOC_BLANK,BRANCH_COUNT,CALL_PAIRS,LOC_CODE_AND_COMMENT,LOC_COMMENTS,CONDITION_COUNT,CYCLOMATIC_COMPLEXITY,CYCLOMATIC_DENSITY,DECISION_COUNT,DECISION_DENSITY,...,NODE_COUNT,NORMALIZED_CYLOMATIC_COMPLEXITY,NUM_OPERANDS,NUM_OPERATORS,NUM_UNIQUE_OPERANDS,NUM_UNIQUE_OPERATORS,NUMBER_OF_LINES,PERCENT_COMMENTS,LOC_TOTAL,defect
0,0.188889,0.053763,0.208333,0.029412,0.102564,0.053763,0.053763,0.029240,0.053763,0.4,...,0.080986,0.048035,0.037776,0.029046,0.038270,0.315789,0.069913,0.335949,0.114286,False
1,0.022222,0.043011,0.125000,0.000000,0.012821,0.043011,0.043011,0.065497,0.032258,0.4,...,0.042254,0.157205,0.009266,0.014226,0.011647,0.368421,0.016230,0.107504,0.042857,False
2,0.022222,0.021505,0.041667,0.014706,0.012821,0.016129,0.021505,0.019883,0.010753,0.6,...,0.017606,0.056769,0.011404,0.016598,0.014975,0.368421,0.027466,0.113201,0.085714,False
3,0.044444,0.021505,0.041667,0.000000,0.000000,0.021505,0.021505,0.035088,0.021505,0.4,...,0.028169,0.082969,0.009266,0.009484,0.016639,0.236842,0.018727,0.000000,0.047619,False
4,0.077778,0.021505,0.041667,0.044118,0.000000,0.000000,0.021505,0.017544,0.000000,0.0,...,0.028169,0.048035,0.018532,0.027267,0.011647,0.184211,0.033708,0.161256,0.095238,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,0.011111,0.032258,0.083333,0.000000,0.000000,0.000000,0.032258,0.046784,0.000000,0.0,...,0.042254,0.135371,0.009266,0.020747,0.009983,0.263158,0.014981,0.000000,0.047619,False
1454,0.133333,0.010753,0.333333,0.000000,0.076923,0.010753,0.010753,0.012865,0.010753,0.4,...,0.042254,0.021834,0.019957,0.023711,0.031614,0.342105,0.047441,0.258009,0.090476,False
1455,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.116959,0.000000,0.0,...,0.000000,0.436681,0.000000,0.001186,0.000000,0.052632,0.000000,0.000000,0.000000,False
1456,0.088889,0.043011,0.000000,0.000000,0.128205,0.000000,0.043011,0.049123,0.000000,0.0,...,0.028169,0.069869,0.014255,0.022525,0.014975,0.526316,0.038702,0.488605,0.057143,False


CM1


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defect
0,0.000237,0.004211,0.013793,0.006452,0.000145,0.000076,1.000000,0.010336,0.004427,6.036150e-07,...,0.0250,0.005900,0.012195,1.0,0.002817,0.003822,0.000159,0.001474,0.002484,False
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000058,0.769231,0.007951,0.003405,4.643192e-07,...,0.0125,0.002950,0.006098,0.5,0.000000,0.003185,0.000000,0.001229,0.000000,True
2,0.054502,0.042105,0.000000,0.032258,0.029894,0.018052,0.084615,0.075535,0.110801,1.363599e-03,...,0.0125,0.000000,0.036585,0.0,0.197183,0.047771,0.034127,0.023342,0.049689,False
3,0.045024,0.031579,0.103448,0.016129,0.022179,0.012584,0.046154,0.127216,0.045866,1.600922e-03,...,0.0000,0.000000,0.018293,0.0,0.211268,0.025478,0.023810,0.019656,0.037267,False
4,0.054502,0.052632,0.172414,0.016129,0.034233,0.020213,0.046154,0.137791,0.067999,2.785720e-03,...,0.0000,0.000000,0.018293,0.0,0.211268,0.038217,0.035714,0.031941,0.062112,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493,0.109005,0.021053,0.000000,0.032258,0.122951,0.091319,0.030769,0.222629,0.190173,2.033064e-02,...,0.0250,0.038348,0.012195,0.0,0.309859,0.146497,0.113492,0.137592,0.024845,True
494,0.054502,0.031579,0.068966,0.032258,0.051109,0.034316,0.038462,0.152103,0.104604,5.219682e-03,...,0.0125,0.020649,0.024390,0.0,0.295775,0.073248,0.052381,0.049140,0.037267,True
495,0.191943,0.105263,0.068966,0.145161,0.228544,0.184290,0.015385,0.355490,0.240364,6.550813e-02,...,0.1125,0.174041,0.213415,0.0,0.436620,0.216561,0.225397,0.233415,0.124224,True
496,0.021327,0.010526,0.000000,0.000000,0.014947,0.008783,0.115385,0.051682,0.078793,4.539603e-04,...,0.0125,0.035398,0.024390,0.0,0.169014,0.041401,0.014286,0.015971,0.012422,True


JM1


,loc,v(g),ev(g),iv(g),n,v,l,d,i,e,...,lOCode,lOComment,lOBlank,locCodeAndComment,uniq_Op,uniq_Opnd,total_Op,total_Opnd,branchCount,defect
0,0.000029,0.000853,0.002439,0.000998,0.000154,0.000016,1.000000,0.003109,0.002282,4.182783e-08,...,0.000708,0.005814,0.004474,0.018519,0.002920,0.001170,0.000221,0.000397,0.000485,False
1,0.000000,0.000000,0.000000,0.000000,0.000118,0.000012,0.769231,0.002391,0.001755,3.217526e-08,...,0.000354,0.002907,0.002237,0.009259,0.002433,0.000975,0.000185,0.000331,0.000000,True
2,0.020634,0.012793,0.000000,0.012469,0.023457,0.014029,0.038462,0.048565,0.098020,7.409672e-04,...,0.018059,0.029070,0.017897,0.009259,0.041363,0.035088,0.020664,0.028467,0.014545,True
3,0.054926,0.004264,0.000000,0.004988,0.071082,0.053793,0.046154,0.040794,0.447313,2.387490e-03,...,0.045680,0.084302,0.062640,0.018519,0.041363,0.131579,0.060701,0.089705,0.004848,True
4,0.010462,0.006397,0.000000,0.007481,0.014927,0.007411,0.046154,0.041105,0.061182,3.313183e-04,...,0.009915,0.002907,0.013423,0.000000,0.026764,0.015595,0.014022,0.016551,0.007273,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10880,0.004940,0.006397,0.000000,0.007481,0.006160,0.002987,0.107692,0.017527,0.057794,5.697788e-05,...,0.004603,0.000000,0.004474,0.000000,0.024331,0.014620,0.005535,0.007282,0.007273,False
10881,0.002325,0.002132,0.000000,0.002494,0.003554,0.001604,0.092308,0.019727,0.027590,3.441723e-05,...,0.001771,0.000000,0.004474,0.000000,0.029197,0.007797,0.003506,0.003641,0.002424,False
10882,0.011915,0.006397,0.000000,0.002494,0.012202,0.006427,0.030769,0.063128,0.034540,4.413390e-04,...,0.010269,0.002907,0.022371,0.000000,0.043796,0.014620,0.010886,0.014565,0.007273,False
10883,0.002616,0.000000,0.000000,0.000000,0.004265,0.001820,0.092308,0.020182,0.030608,3.994783e-05,...,0.002125,0.000000,0.004474,0.000000,0.021898,0.007797,0.003875,0.004965,0.000000,False


# Load Data to 10 Cross Validation

In [13]:
data_load = {}
for i in dataset:
    data_load[i] = tool.load_data(dataset[i])

# SMOTE

In [14]:
def SMOTUNE(X_train, y_train, sampling, neighbors):
    neighbors = int(neighbors)
    sm = SMOTE(random_state=42, sampling_strategy=sampling, k_neighbors=neighbors)
    X_train, y_train = sm.fit_sample(X_train, y_train)
    return X_train, y_train

# Bayesian Optimization

In [15]:
from bayes_opt import UtilityFunction

def optimize(model, pbounds, init = 15, iter = 35):
    repro()
    utility = UtilityFunction(kind="ei", kappa=2.5, xi=0.0)

    optimizer = bayes(
        f=model,
        pbounds=pbounds,
        verbose=2,
        random_state=42,
    )

#     max = 0
#     for i in range(iter):

#         print("\n======== ITERASI ",i," =========")
#         if(i < init):
#             next_point = tool.generate_random_sample(pbounds)
#         else:
#             next_point = optimizer.suggest(utility)


#         target = model(**next_point)
#         optimizer.register(params=next_point, target=target)
        
#         print(target)
#         if(target > max):
#             max = target
#             print("\x1b[31m",target,next_point,"\x1b[0m")
#         else:
#             print(target, next_point)
    optimizer.maximize(init_points=init, n_iter=iter, acq='ei')

    return optimizer

# Tie all it Together

In [16]:
from tensorflow.keras.utils import plot_model
from imblearn.over_sampling import SMOTE
test = []
def run_nn(data, fold, pbounds, do_optimize = True):
    repro()
    fold_generator = tool.split(data, fold)
    sm = SMOTE(random_state=42)

    
    def go(do_optimize = True, **pbounds):
        
        y_train_pred_collect, y_dev_pred_collect, y_test_pred_collect = [], [], []
        y_train_true_collect, y_dev_true_collect, y_test_true_collect = [], [], []
        models = model.neuralNet(**pbounds)
        
        
        
        for remainder, test in fold_generator():
            repro()
            train, dev = next(tool.split(remainder, fold)())

            X_train, y_train = train[0], train[1]
            X_dev, y_dev = dev[0], dev[1]
            X_test, y_test = test[0],test[1]
            
            if 'sampling' and 'neighbors' in pbounds:
                X_train, y_train = SMOTUNE(X_train, y_train, pbounds['sampling'], pbounds['neighbors'])

            models.fit(X_train, y_train, epochs = 15, batch_size = 64, verbose = 0)


            #Collect Train Dev Test
    
            y_train_pred_collect = np.append(y_train_pred_collect, models.predict(X_train).round())
            y_train_true_collect = np.append(y_train_true_collect, y_train)
            
            y_dev_pred_collect = np.append(y_dev_pred_collect, models.predict(X_dev).round())
            y_dev_true_collect = np.append(y_dev_true_collect, y_dev)
            
            y_test_pred_collect = np.append(y_test_pred_collect, models.predict(X_test).round())
            y_test_true_collect = np.append(y_test_true_collect, y_test)

        
        if(do_optimize):
            result = metric.gmeans(y_dev_true_collect, y_dev_pred_collect)
        else:

            result = {
                "Train" : {
                    "Gmeans" : metric.gmeans(y_train_true_collect, y_train_pred_collect),
                    "Recall" : metric.recall(y_train_true_collect, y_train_pred_collect),
                    "pf" : metric.pf(y_train_true_collect, y_train_pred_collect),
                    "cf_matrix" : metric.conf_matrix(y_train_true_collect, y_train_pred_collect)
                },
                "Dev" : {
                    "Gmeans" : metric.gmeans(y_dev_true_collect, y_dev_pred_collect),
                    "Recall" : metric.recall(y_dev_true_collect, y_dev_pred_collect),
                    "pf" : metric.pf(y_dev_true_collect, y_dev_pred_collect),
                    "cf_matrix" : metric.conf_matrix(y_dev_true_collect, y_dev_pred_collect)
                },
                "Test" : {
                    "Gmeans" : metric.gmeans(y_test_true_collect, y_test_pred_collect),
                    "Recall" : metric.recall(y_test_true_collect, y_test_pred_collect),
                    "pf" : metric.pf(y_test_true_collect, y_test_pred_collect),
                    "cf_matrix" : metric.conf_matrix(y_test_true_collect, y_test_pred_collect)
                }
                
            }
        return result
        
    if(do_optimize):
        return optimize(go, pbounds, init = 5, iter = 45)
    else:
        return go(do_optimize = False, **pbounds)
        

# Define Hyperparam Search Space

Learning Rate = 0.001 to 0.01

Dropout = 0.1 to 0.6

layer = 1 to 5

sampling =  DMR to 1

neighbors =  1 - 20

In [17]:
bound = {}
for i in dataset:
    bound[i] = {'lr': (0.001, 0.01),
           'drop': (0.1, 0.6),
           'layer': (1, 5),
           'neurons': (1, 50),
           'sampling': ((dmr_data[i]/100)+0.1,1),
           'neighbors': (1,20)
          }

bound_without_smote = {'lr': (0.001, 0.01),
           'drop': (0.1, 0.6),
           'layer': (1, 5),
           'neurons': (1, 50)
          }

# Let's Optimize it

## With SMOTE

### KC1

In [ ]:
hyperparam_KC1 = run_nn(data_load['KC1'], 10, bound['KC1'])

|   iter    |  target   |   drop    |   layer   |    lr     | neighbors |  neurons  | sampling  |
-------------------------------------------------------------------------------------------------
|  1        |  0.3683   |  0.2873   |  4.803    |  0.007588 |  12.37    |  8.645    |  0.3709   |
|  2        |  0.7129   |  0.129    |  4.465    |  0.00641  |  14.45    |  2.009    |  0.9776   |
|  3        |  0.6941   |  0.5162   |  1.849    |  0.002636 |  4.485    |  15.91    |  0.6458   |
|  4        |  0.6677   |  0.316    |  2.165    |  0.006507 |  3.65     |  15.32    |  0.5277   |
|  5        |  0.6135   |  0.328    |  4.141    |  0.002797 |  10.77    |  30.03    |  0.2892   |
|  6        |  0.4574   |  0.1      |  4.125    |  0.00522  |  16.53    |  1.0      |  1.0      |
|  7        |  0.7054   |  0.1395   |  4.591    |  0.006853 |  13.68    |  2.367    |  0.9711   |
|  8        |  0.0      |  0.145    |  3.785    |  0.006213 |  13.49    |  1.319    |  0.8181   |
|  9        |  0.702

In [ ]:
best_hyperparam_KC1 = hyperparam_KC1.max['params']
result_KC1 = run_nn(data_load['KC1'], 10, best_hyperparam_KC1, do_optimize = False)
result_KC1

### KC2

In [ ]:
hyperparam_KC2 = run_nn(data_load['KC2'], 10, bound['KC2'])

In [ ]:
best_hyperparam_KC2 = hyperparam_KC2.max['params']
result_KC2 = run_nn(data_load['KC2'], 10, best_hyperparam_KC2, do_optimize = False)
result_KC2

In [ ]:
def ensure_rng(random_state=None):
    """
    Creates a random number generator based on an optional seed.  This can be
    an integer or another random state for a seeded rng, or None for an
    unseeded rng.
    """
    if random_state is None:
        random_state = np.random.RandomState()
    elif isinstance(random_state, int):
        random_state = np.random.RandomState(random_state)
    else:
        assert isinstance(random_state, np.random.RandomState)
    return random_state

In [ ]:
random_state = ensure_rng(random_state=42)
x_seeds = random_state.uniform(bound['KC1'][:, 0], bounds['KC1'][:, 1],size=(100, bound['KC1'].shape[0]))

### PC1

In [ ]:
hyperparam_PC1 = run_nn(data_load['PC1'], 5, bound['PC1'])

In [ ]:
best_hyperparam_PC1 = hyperparam_PC1.max['params']
result_PC1 = run_nn(data_load['PC1'], 5, best_hyperparam_PC1, do_optimize = False)
result_PC1

### PC3

In [ ]:
hyperparam_PC3 = run_nn(data_load['PC3'], 7, bound['PC3'])

In [ ]:
best_hyperparam_PC3 = hyperparam_PC3.max['params']
result_PC3 = run_nn(data_load['PC3'], 7, best_hyperparam_PC3, do_optimize = False)
result_PC3

### PC4

In [ ]:
hyperparam_PC4 = run_nn(data_load['PC4'], 7, bound['PC4'])

In [ ]:
best_hyperparam_PC4 = hyperparam_PC4.max['params']
result_PC4 = run_nn(data_load['PC4'], 7, best_hyperparam_PC4, do_optimize = False)
result_PC4

## CM1

In [ ]:
hyperparam_CM1 = run_nn(data_load['CM1'], 5, bound['CM1'])

In [ ]:
best_hyperparam_CM1 = hyperparam_CM1.max['params']
result_CM1 = run_nn(data_load['CM1'], 5, best_hyperparam_CM1, do_optimize = False)
result_CM1

## JM1

In [ ]:
hyperparam_JM1 = run_nn(data_load['JM1'], 10, bound['JM1'])

In [ ]:
best_hyperparam_JM1 = hyperparam_JM1.max['params']
result_JM1 = run_nn(data_load['JM1'], 10, best_hyperparam_JM1, do_optimize = False)
result_JM1

## Without SMOTE

### KC1

In [ ]:
hyperparam_KC1_wihout_SMOTE = run_nn(data_load['KC1'], 10, bound_without_smote)

In [ ]:
best_hyperparam_KC1_without_SMOTE = hyperparam_KC1_wihout_SMOTE.max['params']
result_KC1_without_SMOTE = run_nn(data_load['KC1'], 10, best_hyperparam_KC1_without_SMOTE, do_optimize = False)
result_KC1_without_SMOTE

### KC2

In [ ]:
hyperparam_KC2_wihout_SMOTE = run_nn(data_load['KC2'], 10, bound_without_smote)

In [ ]:
best_hyperparam_KC2_without_SMOTE = hyperparam_KC2_wihout_SMOTE.max['params']
result_KC2_without_SMOTE = run_nn(data_load['KC2'], 10, best_hyperparam_KC2_without_SMOTE, do_optimize = False)
result_KC2_without_SMOTE

### PC1

In [ ]:
hyperparam_PC1_wihout_SMOTE = run_nn(data_load['PC1'], 5, bound_without_smote)

In [ ]:
best_hyperparam_PC1_without_SMOTE = hyperparam_PC1_wihout_SMOTE.max['params']
result_PC1_without_SMOTE = run_nn(data_load['PC1'], 5, best_hyperparam_PC1_without_SMOTE, do_optimize = False)
result_PC1_without_SMOTE

### PC3

In [ ]:
hyperparam_PC3_wihout_SMOTE = run_nn(data_load['PC3'], 7, bound_without_smote)

In [ ]:
best_hyperparam_PC3_without_SMOTE = hyperparam_PC3_wihout_SMOTE.max['params']
result_PC3_without_SMOTE = run_nn(data_load['PC3'], 7, best_hyperparam_PC3_without_SMOTE, do_optimize = False)
result_PC3_without_SMOTE

### PC4

In [ ]:
hyperparam_PC4_wihout_SMOTE = run_nn(data_load['PC4'], 7, bound_without_smote)

In [ ]:
best_hyperparam_PC4_without_SMOTE = hyperparam_PC4_wihout_SMOTE.max['params']
result_PC4_without_SMOTE = run_nn(data_load['PC4'], 7, best_hyperparam_PC4_without_SMOTE, do_optimize = False)
result_PC4_without_SMOTE

# Result

In [ ]:
result_PC4_without_SMOTE

## CM1

In [ ]:
hyperparam_CM1_wihout_SMOTE = run_nn(data_load['CM1'], 5, bound_without_smote)

In [ ]:
best_hyperparam_CM1_without_SMOTE = hyperparam_CM1_wihout_SMOTE.max['params']
result_CM1_without_SMOTE = run_nn(data_load['CM1'], 5, best_hyperparam_CM1_without_SMOTE, do_optimize = False)
result_CM1_without_SMOTE

## JM1

In [ ]:
hyperparam_JM1_wihout_SMOTE = run_nn(data_load['JM1'], 10, bound_without_smote)

In [ ]:
best_hyperparam_JM1_without_SMOTE = hyperparam_JM1_wihout_SMOTE.max['params']
result_JM1_without_SMOTE = run_nn(data_load['JM1'], 10, best_hyperparam_JM1_without_SMOTE, do_optimize = False)
result_JM1_without_SMOTE

## Store Variable

In [ ]:
from datetime import datetime
import dill

dill.dump_session('Save/skripsi - EI - matrix '+ datetime.now().strftime('%Y-%m-%d %H %M %S')+ '.db')

In [ ]:
import dill
dill.load_session('Save/skripsi - EI - matrix-final 2020-10-31 17 18 59.db')